In [1]:
%matplotlib inline
import pandas as pd

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [7]:
df0 = pd.read_csv('heart.csv')

In [8]:
df0.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [9]:
# Seleciona colunas e remove os NaNs
df1 = df0[['Age','Sex','ExerciseAngina','HeartDisease']].dropna()
df = df1.rename(columns={'Age': 'idade', 'Sex': 'sexo', 'ExerciseAngina': 'exercicio_fisico', 'HeartDisease': 'doencas_cardiacas' })
df.head()

,idade,sexo,exercicio_fisico,doencas_cardiacas
0,40,M,N,0
1,49,F,N,1
2,37,M,N,0
3,48,F,Y,1
4,54,M,N,0


In [ ]:
# M é masculino e F é feminino, N é não faz exercicio e Y é quem faz, 0 é quem não tem doenças cardíacas e 1 é quem tem